In [1]:
def geo_to_json(capital, lat, lon):
    return {
        'capital': capital,
        'latitude': lat,
        'longitude': lon
    }
    
def read_capitals(filename):
    # Read data from the file 'capitals_positions.txt'
    with open(filename, 'r') as file:
        lines = file.readlines()
    
    # Process the data and add nodes and edges to the graph
    for line in lines:
        line = line.strip().split(',')
        
        city = line[0]
        country = line[1]
        
        lat = float(line[2])
        lon = float(line[3])
        
        yield city, country, lat, lon
        


In [3]:
# Time spent per location
SOLVER_TIME_LIMIT=30

# Time spent per location
VEHICLE_LOCATION_TIME_LIMIT=100

# maximum time per each location
MAX_ROUTING_TIME_LIMIT=100000

# don't force start cumul to zero
IS_CUMUL_ZERO=False

# no slack
SLACK_AMOUNT=0

DISTANCE_METHOD='geographical'

RADIUS=6371
DISTANCE_CONFIG={'radius': RADIUS}

In [ ]:
from ortools.linear_solver import pywraplp
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


# Define the geographical coordinates and names of locations
locations = {
    "Kabul": (34.52933, 69.12532),
    "Tirana": (41.32755, 19.8187),
    "Algiers": (36.75377, 3.05876),
    "Andorra la Vella": (42.50729, 1.5218),
    "Luanda": (-8.81155, 13.242),
    "Buenos Aires": (-34.60372, -58.38159),
    "Yerevan": (40.18111, 44.51361),
}

num_locations = len(locations)
distance_matrix = [[0] * num_locations for _ in range(num_locations)]

for i in range(num_locations):
    for j in range(num_locations):
        if i != j:
            points_keys=list(locations.keys())
            
            P_i=locations[points_keys[i]]
            P_j=locations[points_keys[j]]
            
            distance_matrix[i][j] = haversine(P_i, P_j)

# Create a routing model
def create_routing_model(distance_matrix, depot, num_vehicles):
    tsp_size = len(distance_matrix)

    manager = pywrapcp.RoutingIndexManager(tsp_size, num_vehicles, depot)
    routing = pywrapcp.RoutingModel(manager)

    # Set the cost function (distance matrix)
    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        
        return distance_matrix[from_node][to_node]
    
    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Set the maximum time allowed per each visit
    dimension_name="Time"
    
    routing.AddDimension(
        transit_callback_index,
        SLACK_AMOUNT,             
        MAX_ROUTING_TIME_LIMIT,
        IS_CUMUL_ZERO,
        dimension_name
    )
    
    # Set a time limit per each location
    time_dimension = routing.GetDimensionOrDie(dimension_name)
    time_dimension.SetSpanUpperBoundForVehicle(LOCATION_TIME_LIMIT)

    return routing, manager

# Solve the problem
def solve_routing_problem():
    # Number of vehicles (1 for a basic TSP)
    num_vehicles = 1
    
    # The depot is the first location in the list
    depot = 0
    
    routing, search_parameters = create_routing_model(distance_matrix, num_vehicles, depot)
    
    # Set a time limit for the solver
    search_parameters.time_limit.seconds = SOLVER_TIME_LIMIT
    
    solution = routing.SolveWithParameters(search_parameters)
    
    if solution:
        print_solution(routing, solution)

# Print the solution
def print_solution(routing, solution):
    print("Objective: {} km".format(solution.ObjectiveValue()))
    
    index = routing.Start(0)
    plan_output = "Route:\n"
    route_distance = 0
    
    while not routing.IsEnd(index):
        node=routing.IndexToNode(index)
        route=list(locations.keys())[node]
        value=solution.Value(routing.NextVar(index))
        
        plan_output += " -> {} ({} km)\n".format(route, value)
        previous_index = index
        
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    
    print(plan_output)
    print("Total Distance: {} km".format(route_distance))

if __name__ == "__main__":
    solve_routing_problem()

In [4]:
import networkx as nx
from spycio import distance

capitals_filename='./capital_positions.txt'

def geo_to_json(capital, lat, lon):
    return {
        'capital': capital,
        'latitude': lat,
        'longitude': lon
    }
    
def read_capitals(filename):
    # Read data from the file 'capitals_positions.txt'
    with open(filename, 'r') as file:
        lines = file.readlines()
    
    # Process the data and add nodes and edges to the graph
    for line in lines:
        line = line.strip().split(',')
        
        city = line[0]
        country = line[1]
        
        lat = float(line[2])
        lon = float(line[3])
        
        yield city, country, lat, lon

# Create a geographical dictionary 
geo_dict={}

# Create a graph
G = nx.Graph()

# Process the data and add nodes and edges to the graph
for city, country, lat, lon in read_capitals(capitals_filename):
    geo_dict[country]=geo_to_json(city, lat, lon)      
    G.add_node(city, lat=lat, lon=lon)

# Calculate the distances and add weighted edges to the graph
for city1, data1 in G.nodes(data=True):
    for city2, data2 in G.nodes(data=True):
        if city1 != city2:
            lat1, lon1 = data1['lat'], data1['lon']
            lat2, lon2 = data2['lat'], data2['lon']
            
            A=[lat1, lon1]
            B=[lat2, lon2]
            
            dist = distance(A, B, DISTANCE_METHOD, DISTANCE_CONFIG)
            
            G.add_edge(city1, city2, weight=dist)

# Calculate the shortest path between two capitals
start = "Kabul"
end = "Vienna"

shortest_path = nx.shortest_path(G, source=start, target=end, weight="weight")
shortest_distance = nx.shortest_path_length(G, source=start, target=end, weight="weight")

print("Shortest path:", shortest_path)
print("Shortest distance:", shortest_distance)

Shortest path: ['Kabul', 'Vienna']
Shortest distance: 4557.0648423953135


In [ ]:
# Calculate the distances and add weighted edges to the graph
for city1, data1 in G.nodes(data=True):
    for city2, data2 in G.nodes(data=True):
        if city1 != city2:
            lat1, lon1 = data1['lat'], data1['lon']
            lat2, lon2 = data2['lat'], data2['lon']
            distance = ((lat1 - lat2) ** 2 + (lon1 - lon2) ** 2) ** 0.5  # Euclidean distance
            G.add_edge(city1, city2, weight=distance)

# Calculate the shortest path between two capitals
start = "Kabul"
end = "Vienna"
shortest_path = nx.shortest_path(G, source=start, target=end, weight="weight")
shortest_distance = nx.shortest_path_length(G, source=start, target=end, weight="weight")

print("Shortest path:", shortest_path)
print("Shortest distance:", shortest_distance)